In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df_train['LotFrontage'] = df_train.groupby('SalePrice')['LotFrontage'].transform(lambda x: x.fillna(x.mean()))
df_train['MasVnrArea'] = df_train.groupby('SalePrice')['MasVnrArea'].transform(lambda x: x.fillna(x.mean()))

In [4]:
df_train['train']  = 1
df_test['train']  = 0
df = pd.concat([df_train, df_test], axis=0,sort=False)
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,train
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0,1
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0,1
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0,1
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0,1
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN,0
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN,0
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN,0
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN,0


# Data Wrangling

In [5]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(50)

,Total,Percent
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
SalePrice,1459,0.499829
FireplaceQu,1420,0.486468
LotFrontage,301,0.103118
GarageYrBlt,159,0.054471
GarageQual,159,0.054471
GarageFinish,159,0.054471


In [6]:
# df = df.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1)

In [7]:
df_no_TotalBsmtSF = df[~df['TotalBsmtSF'].isna()]

In [8]:
TotalBsmtSF_ratio=df_no_TotalBsmtSF['TotalBsmtSF'].sum()/(df_no_TotalBsmtSF['TotalBsmtSF'].sum() + df_no_TotalBsmtSF['1stFlrSF'].sum() + df_no_TotalBsmtSF['2ndFlrSF'].sum())
TotalBsmtSF_ratio

0.41277766883661327

In [9]:
df['TotalBsmtSF']= df['TotalBsmtSF'].fillna(0.7*(df['1stFlrSF'] + df['2ndFlrSF']))

In [10]:
df_non_LotFrontage = df[~df['LotFrontage'].isna()]

In [11]:
LotFrontage_perc=df_non_LotFrontage['LotFrontage'].sum()/(df_non_LotFrontage['TotalBsmtSF'].sum() + df_non_LotFrontage['1stFlrSF'].sum() + df_non_LotFrontage['2ndFlrSF'].sum())
LotFrontage_perc

0.027395640835032136

In [12]:
df['TotalSA']=df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

In [13]:
df['LotFrontage']= df['LotFrontage'].fillna(0.027*df['TotalSA'])

In [14]:
df_no_MasVnrArea = df[~df['MasVnrArea'].isna()]
MasVnrArea_perc=df_no_MasVnrArea['MasVnrArea'].sum()/(df_no_MasVnrArea['TotalBsmtSF'].sum() + df_no_MasVnrArea['1stFlrSF'].sum() + df_no_MasVnrArea['2ndFlrSF'].sum())
MasVnrArea_perc

0.04015368472464079

In [15]:
df['MasVnrArea']= df['MasVnrArea'].fillna(0.04*df['TotalSA'])

In [16]:
df_no_GarageYrBlt = df[~df['GarageYrBlt'].isna()]
diff = (df_no_GarageYrBlt['YearBuilt'].sum() - df_no_GarageYrBlt['GarageYrBlt'].sum())/len(df_no_GarageYrBlt['YearBuilt'])
diff

-5.225724637681159

In [17]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['YearBuilt']+5.23)

In [18]:
df_no_BsmtUnfSF = df[~df['BsmtUnfSF'].isna()]

In [19]:
BsmtUnfSF_ratio=df_no_BsmtUnfSF['BsmtUnfSF'].sum()/df_no_BsmtUnfSF['TotalBsmtSF'].sum()
BsmtUnfSF_ratio

0.5331660523145808

In [20]:
df['BsmtUnfSF'] = df['BsmtUnfSF'].fillna(0.533*df['TotalBsmtSF'])

In [21]:
df_no_BsmtFinSF1 = df[~df['BsmtFinSF1'].isna()]
BsmtFinSF1_ratio=df_no_BsmtFinSF1['BsmtFinSF1'].sum()/df_no_BsmtFinSF1['TotalBsmtSF'].sum()
BsmtFinSF1_ratio

0.419692566551551

In [22]:
df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna(0.42*df['TotalBsmtSF'])

In [23]:
df_no_BsmtFinSF2 = df[~df['BsmtFinSF2'].isna()]
BsmtFinSF2_ratio=df_no_BsmtFinSF2['BsmtFinSF2'].sum()/df_no_BsmtFinSF2['TotalBsmtSF'].sum()
BsmtFinSF2_ratio

0.04714138113386815

In [24]:
df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(0.047*df['TotalBsmtSF'])

In [25]:
df['BsmtFullBath'] = df['BsmtFullBath'].fillna(df['BsmtFullBath'].median())

In [26]:
df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(df['BsmtHalfBath'].median())

In [27]:
df['GarageCars'] = df.groupby('TotalBsmtSF')['GarageCars'].transform(lambda x: x.fillna(x.mean()))
df['GarageArea'] = df.groupby('TotalBsmtSF')['GarageArea'].transform(lambda x: x.fillna(x.mean()))

In [28]:
df.LotFrontage.fillna(df.LotFrontage.median(), inplace=True)
df.GarageYrBlt.fillna(0, inplace=True)
df.MasVnrArea.fillna(0, inplace=True)

In [29]:
# Filling missing values for categorical features
df.PoolQC.fillna('NA', inplace=True)
df.MiscFeature.fillna('NA', inplace=True)    
df.Alley.fillna('NA', inplace=True)          
df.Fence.fillna('NA', inplace=True)         
df.FireplaceQu.fillna('NA', inplace=True)    
df.GarageCond.fillna('NA', inplace=True)    
df.GarageQual.fillna('NA', inplace=True)     
df.GarageFinish.fillna('NA', inplace=True)   
df.GarageType.fillna('NA', inplace=True)     
df.BsmtExposure.fillna('NA', inplace=True)     
df.BsmtCond.fillna('NA', inplace=True)        
df.BsmtQual.fillna('NA', inplace=True)        
df.BsmtFinType2.fillna('NA', inplace=True)     
df.BsmtFinType1.fillna('NA', inplace=True)     
df.MasVnrType.fillna('None', inplace=True)   
df.Exterior2nd.fillna('None', inplace=True) 

# These are general properties that all houses should have, so NANs probably 
# just mean the values were not recorded. I therefore use "mode", the most 
# common value to fill in
df.Functional.fillna(df.Functional.mode()[0], inplace=True)       
df.Utilities.fillna(df.Utilities.mode()[0], inplace=True)          
df.Exterior1st.fillna(df.Exterior1st.mode()[0], inplace=True)        
df.SaleType.fillna(df.SaleType.mode()[0], inplace=True)                
df.KitchenQual.fillna(df.KitchenQual.mode()[0], inplace=True)        
df.Electrical.fillna(df.Electrical.mode()[0], inplace=True)    

# MSZoning should highly correlate with the location, so I use the mode values of individual 
# Neighborhoods
for i in df.Neighborhood.unique():
    if df.MSZoning[df.Neighborhood == i].isnull().sum() > 0:
        df.loc[df.Neighborhood == i,'MSZoning'] = \
        df.loc[df.Neighborhood == i,'MSZoning'].fillna(df.loc[df.Neighborhood == i,'MSZoning'].mode()[0])

In [30]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(50)

,Total,Percent
SalePrice,1459,0.499829
Id,0,0.000000
KitchenAbvGr,0,0.000000
GarageFinish,0,0.000000
GarageYrBlt,0,0.000000
GarageType,0,0.000000
FireplaceQu,0,0.000000
Fireplaces,0,0.000000
Functional,0,0.000000
TotRmsAbvGrd,0,0.000000


# One Hot Encoding

In [31]:
# These categorical features are "rank", so they can be transformed to 
# numerical features
df.Alley = df.Alley.map({'NA':0, 'Grvl':1, 'Pave':2})
df.BsmtCond =  df.BsmtCond.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.BsmtExposure = df.BsmtExposure.map({'NA':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4})
df['BsmtFinType1'] = df['BsmtFinType1'].map({'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6})
df['BsmtFinType2'] = df['BsmtFinType2'].map({'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6})
df.BsmtQual = df.BsmtQual.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.ExterCond = df.ExterCond.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.ExterQual = df.ExterQual.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.FireplaceQu = df.FireplaceQu.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.Functional = df.Functional.map({'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8})
df.GarageCond = df.GarageCond.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.GarageQual = df.GarageQual.map({'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.HeatingQC = df.HeatingQC.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.KitchenQual = df.KitchenQual.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
df.LandSlope = df.LandSlope.map({'Sev':1, 'Mod':2, 'Gtl':3}) 
df.PavedDrive = df.PavedDrive.map({'N':1, 'P':2, 'Y':3})
df.PoolQC = df.PoolQC.map({'NA':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
df.Street = df.Street.map({'Grvl':1, 'Pave':2})
df.Utilities = df.Utilities.map({'ELO':1, 'NoSeWa':2, 'NoSewr':3, 'AllPub':4})

df.MSSubClass = df.MSSubClass.map({20:'class1', 30:'class2', 40:'class3', 45:'class4',
                                   50:'class5', 60:'class6', 70:'class7', 75:'class8',
                                   80:'class9', 85:'class10', 90:'class11', 120:'class12',
                                   150:'class13', 160:'class14', 180:'class15', 190:'class16'})

In [32]:
df=df.drop(['TotalSA'],axis=1)
qual = list( df.loc[:,df.dtypes == 'object'].columns.values )

In [33]:
# create of list of dummy variables that I will drop, which will be the last
# column generated from each categorical feature
dummy_drop = []
for i in qual:
    dummy_drop += [ i+'_'+str(df[i].unique()[-1]) ]

# create dummy variables
df = pd.get_dummies(df,columns=qual) 
# drop the last column generated from each categorical feature
df = df.drop(dummy_drop,axis=1)

# Prediction

In [34]:
df = df.drop(['Id',],axis=1)
df_train = df[df['train'] == 1]
df_train = df_train.drop(['train',],axis=1)


df_test = df[df['train'] == 0]
df_test = df_test.drop(['SalePrice'],axis=1)
df_test = df_test.drop(['train',],axis=1)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('SalePrice', axis=1), df_train['SalePrice'], test_size=0.25, random_state=42)

In [36]:
from sklearn.model_selection import KFold, cross_val_score
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

In [37]:
from sklearn.metrics import make_scorer, mean_squared_error
scorer = make_scorer(mean_squared_error, greater_is_better = False)

def rmse_cv(model):
    kf = KFold(5, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring = scorer, cv = kf))
    return(rmse)

from sklearn.metrics import make_scorer, mean_squared_error
scorer = make_scorer(mean_squared_error, greater_is_better = False)

def rmse_cv(model):
    kf = KFold(5, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring = scorer, cv = kf))
    return(rmse)

In [38]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 100, random_state = 42)
score = rmse_cv(rfr)
print('Random Forest score = {:.4f}  (std = {:.4f})'.format(score.mean(), score.std()))

Random Forest score = 32385.3368  (std = 8101.0658)


In [39]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
score = rmse_cv(lr)
print('Linear Regression score = {:.4f}  (std = {:.4f})'.format(score.mean(), score.std()))

Linear Regression score = 48473.1589  (std = 11906.0721)


In [40]:
from sklearn.tree import DecisionTreeRegressor
dtreg = DecisionTreeRegressor(random_state = 42)
score = rmse_cv(dtreg)
print('Decision Tree score = {:.4f}  (std = {:.4f})'.format(score.mean(), score.std()))

Decision Tree score = 44414.2834  (std = 5019.9898)


In [41]:
import xgboost as xgb
xgbr = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
        learning_rate=0.05, max_depth=3, min_child_weight=1.7817,
        n_estimators=2200, reg_alpha=0.4640, reg_lambda=0.8571,
        subsample=0.5213, silent=True, random_state =7, nthread = -1)
score = rmse_cv(xgbr)
print('XGBoost score = {:.4f}  (std = {:.4f})'.format(score.mean(), score.std()))

[01:05:08] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:05:12] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:05:17] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:05:22] WARNING: ..

In [43]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=3000, 
            learning_rate=0.05, max_depth=4, max_features='sqrt',
            min_samples_leaf=15, min_samples_split=10, loss='huber',
            random_state=5)
score = rmse_cv(gbr)
print('Gradient Boosting score = {:.4f}  (std = {:.4f})'.format(score.mean(), score.std()))

Gradient Boosting score = 29258.4676  (std = 8530.4883)


In [44]:
gbr.fit(df_train.drop('SalePrice', axis=1), df_train['SalePrice'])

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='huber',
                          max_depth=4, max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=15, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=5, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [45]:
predict_y = gbr.predict(df_test)
predict_y

array([123044.81219533, 159566.19633974, 190564.60001742, ...,
       178712.55490918, 116591.42199296, 203937.03192794])

In [46]:
test = pd.read_csv('test.csv')

In [47]:
submission = pd.DataFrame({
        "Id": test["Id"],
        "SalePrice": predict_y
    })
submission.to_csv('submission_4.csv', index=False)